# Proyecto 2 - Encuesta Multiproposito

**Integrantes**
- Juan Sebastian Sanchez Delgado - 202013577
- Juanita Ovalle
- Miguel Angel Cardenas - 202010998
- Karol Valentina Chaves
- Santiago Paeres Gonzalez - 201425176


## 1. Preparacion de datos
En esta etapa se analizan las caracteristicas principales del conjunto de datos, asi como también las dimensiones de calidad de estos (completitud, unicidad, consistencia, validez). Posteriormente, se limpian los datos y se transforman para la futura etapa de modelamiento.

In [4]:
# Se importan las librerias a usar y se espcifica un numero maximo de filas y columnas
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 50)

In [5]:
# Se cargan los datos

# Encuesta año 2017:
df_encuesta_2017 = pd.read_csv("./DATOS/Proyecto 2 - BI - Asma DatosAjus/Datos_proyecto_II_BI_2017.csv", encoding='utf-8')
df_encuesta_sin_asma_2017 = pd.read_csv("./DATOS/DatosMultipropositoSinAsma-Proy2/Datos_proyecto_II_BI_2017_sin_asma.csv", encoding='latin-1')
# Combinar los DataFrames a lo largo de las filas (axis=0)
df_encuesta_2017 = pd.concat([df_encuesta_2017, df_encuesta_sin_asma_2017], axis=0)

# Encuesta año 2021:
df_encuesta_2021 = pd.read_csv("./DATOS/Proyecto 2 - BI - Asma DatosAjus/Datos_proyecto_II_BI_2021.csv", encoding='utf-8', sep=";")
df_encuesta_sin_asma_2021 = pd.read_csv("./DATOS/DatosMultipropositoSinAsma-Proy2/Datos_proyecto_II_BI_2021_sin_asma.csv", encoding='latin-1')

df_encuesta_2021 = pd.concat([df_encuesta_2021, df_encuesta_sin_asma_2021], axis=0)

/var/folders/lf/7dykkp1x3jscxj8qjtjfxs4h0000gn/T/ipykernel_80229/3571502707.py:4: DtypeWarning: Columns (145,295,297,380,381,383,469,495,496,499,500,501,508,509,510,511) have mixed types. Specify dtype option on import or set low_memory=False.
  df_encuesta_2017 = pd.read_csv("./DATOS/Proyecto 2 - BI - Asma DatosAjus/Datos_proyecto_II_BI_2017.csv", encoding='utf-8')
/var/folders/lf/7dykkp1x3jscxj8qjtjfxs4h0000gn/T/ipykernel_80229/3571502707.py:5: DtypeWarning: Columns (12,145,146,147,280,285,295,296,297,304,379,380,381,382,383,469,470,495,496,498,499,500,501,507,508,509,510,511,512,513) have mixed types. Specify dtype option on import or set low_memory=False.
  df_encuesta_sin_asma_2017 = pd.read_csv("./DATOS/DatosMultipropositoSinAsma-Proy2/Datos_proyecto_II_BI_2017_sin_asma.csv", encoding='latin-1')


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 57: invalid continuation byte

In [ ]:
df_encuesta_2017.shape

In [ ]:
df_encuesta_2021.shape

Como se puede ver la encuesta realizada en el año 2021 tiene una población más grande que la se hizo en el año 2017.

In [ ]:
df_encuesta_2017.dtypes

In [ ]:
df_encuesta_2021.dtypes

La mayoria de las preguntas de la encuesta se responden de forma numerica. Es decir son de selección multiple.

In [ ]:
df_encuesta_2017.head(15)

In [ ]:
df_encuesta_2021.head(15)

Ahora dentro del conjunto completo de datos sobre la Encuesta Multipropósito, se escogio un subconjunto de datos con las preguntas que son de interes para resolver los requerimientos analiticos planteados. Dentro de estas preguntas que se consideran de interes se incluyen:

- Año 2017:

    1. **LOCALIDAD_TEX**: Nombre de la localidad

    2. **DPTOMPIO**: Divipola departamento-municipio
        
    3. **NPCFP14I**: A ... le han diagnosticado alguna de estas enfermedades o problemas de salud: 9. Asma? 

- Año 2021:
    1. **DPTO**: Departamento

    2. **MPIO**: Municipio

    3. **NOMBRE_LOCALIDAD**: Nombre de la localidad 

    4. **NOMBRE_ESTRATO**: Nombre estrato de muestreo

    5. **NVCBP15I**: ¿Cuáles de los siguientes problemas presenta el entorno donde está ubicada su vivienda?: 9. Contaminación de cuerpos de agua (ríos, caños, humedales, lagunas)

    6. **NPCFP14F**: ¿A ...le han diagnosticado alguna de estas enfermedades o problemas de salud: 6. Enfermedades respiratorias crónicas como: Asma, EPOC, Enfisema?

- Preguntas de interes para ambas encuentas:

    1. **NVCBP11AA**: Estrato por tarifa de servicios publicos.

    2. **NPCFP14H**: A ... le han diagnosticado alguna de estas enfermedades o problemas de salud: 8. ¿Enfermedades mentales, trastornos de la conciencia (depresión, ansiedad)? 

    3. **NPCFP36**: En los ÚLTIMOS 30 DÍAS, ¿cuántas veces por semana ... practicó deporte o realizó actividad física por 30 minutos continuos o más?

    4. **NVCBP9**: ¿Algún espacio de la edificación donde está ubicada la vivienda está dedicado a negocios de industria, comercio o servicios? 

    5. **NVCBP12**: ¿Cuál es el material predominante de las paredes exteriores? 

    6. **NVCBP13**: ¿Cuál es el material predominante de los pisos? 

    7. **NVCBP14A**: La vivienda está cerca de:1. Fábricas o industrias 

    8. **NPCFP14B**: A ... le han diagnosticado alguna de estas enfermedades o problemas de salud: 2. Enfermedades respiratorias, bronquitis, enfisema? 

    9. **NVCBP14I**: La vivienda está cerca de:9. Caños de aguas residuales

    10. **NVCBP15D**: ¿Cuáles de los siguientes problemas presenta el sector donde está ubicada su vivienda?4. Contaminación del aire

    11. **NVCBP8A**: ¿La vivienda presenta:1. Humedades en el techo o en las paredes? 
        
    12. **NVCBP8E**:¿La vivienda presenta:5. Grietas en el piso? 

    13. **NVCBP8B**:¿La vivienda presenta:2. Goteras en el techo?

    14. **NVCBP4**: ¿La edificación está ubicada en un conjunto cerrado?

    15. **NVCBP8D**:¿La vivienda presenta:4. Fallas en tuberías, cañerías o desagües?

    16. **NVCBP8C**:¿La vivienda presenta:3. Grietas en techos y paredes?

    17. **NVCBP8E**:¿La vivienda presenta:5. Grietas en el piso?

    18. **NVCBP8G**:¿La vivienda presenta:7. Escasa ventilación?

    19. **NVCBP10**:Tipo de vivienda

In [ ]:
# Constantes utiles:
COLUMNA_ASMA_2017 = 'NPCFP14I'

COLUMNA_ASMA_2021 = 'NPCFP14F'

# Se filtra el Dataframe por las columnas de interes
preguntas_interes_2017 = ['LOCALIDAD_TEX', 'DPTOMPIO', COLUMNA_ASMA_2017]

preguntas_interes_2021 = ['DPTO', 'MPIO', 'NOMBRE_LOCALIDAD', 'NOMBRE_ESTRATO', 'NVCBP15I', COLUMNA_ASMA_2021]

preguntas_comunes = [
    'NVCBP11AA', 'NPCFP14H', 'NPCFP36', 'NVCBP9',
    'NVCBP12', 'NVCBP13', 'NVCBP14A', 'NPCFP14B', 'NVCBP14I',
    'NVCBP15D', 'NVCBP8A', 'NVCBP8B', 'NVCBP4',
    'NVCBP8D', 'NVCBP8C', 'NVCBP8E', 'NVCBP8G', 'NVCBP10'
]

df_preguntas_interes_2017 = df_encuesta_2017[preguntas_interes_2017+preguntas_comunes]

df_preguntas_interes_2017

In [ ]:
df_preguntas_interes_2021 = df_encuesta_2021[preguntas_interes_2021+preguntas_comunes]

df_preguntas_interes_2021

In [ ]:
df_preguntas_interes_2017.shape

In [ ]:
df_preguntas_interes_2017.dtypes

Todas las respuestas a las preguntas de interes seleccionadas son de tipo numerico, como se puede ver en la tabla superior.

### 1.1 Completitud

In [ ]:
# Se muestra la proporcion de registros con valores nulos con respecto a la totalidad de registros
df_preguntas_interes_2017.isnull().sum() / df_preguntas_interes_2017.shape[0]

Se encuentran aproximadamente 32.7% de los valores nulos para las localidades. Del mismo modo, se registran algunos porcentajes de valores nulos en algunas preguntas significativas.  Por ejemplo, en NPCFP36, es aproximadamente del 14.5%. En caso de no saber qué hacer con los registros nulos y, ya que no representan una porcentanje tan alto del total de los datos, se opta por borrar los valores nulos.

In [ ]:
# Eliminar todos los valores nulos en el DataFrame
df_preguntas_interes_2017 = df_preguntas_interes_2017.dropna()

In [ ]:
df_preguntas_interes_2017.isnull().sum() / df_preguntas_interes_2017.shape[0]

Como se puede ver los regitros nulos (NaN) fueron exitosamente eliminados depues de aplicar la trasformación. 

Ahora se realiza el mismo procedimiento para la encuesta del 2021:

In [ ]:
# Se muestra la proporcion de registros con valores nulos con respecto a la totalidad de registros
df_preguntas_interes_2021.isnull().sum() / df_preguntas_interes_2021.shape[0]

Algunos valores referentes a la ubicacion del encuestado como COD_LOCALIDAD, NOMBRE_LOCALIDAD, y COD_UPZ_GRUPO tienen porcentajes de valores nulos de 16.12%, 16.12% y 18.22% respectivamente. Al igual que con la encuesta de 2017 se procede a borrar estos registros, 

In [ ]:
# Eliminar todos los valores nulos en el DataFrame
df_preguntas_interes_2021 = df_preguntas_interes_2021.dropna()

In [ ]:
# Se muestra la proporcion de registros con valores nulos con respecto a la totalidad de registros
df_preguntas_interes_2021.isnull().sum() / df_preguntas_interes_2021.shape[0]

Como se puede ver los regitros nulos (NaN) fueron exitosamente eliminados depues de aplicar la trasformación.

### 1.2 Unicidad

In [ ]:
# Se revisa el número de filas duplicadas
# Calcular el porcentaje de filas duplicadas
porcentaje_duplicadas_2017 = (df_preguntas_interes_2017.duplicated(keep=False).sum() / df_preguntas_interes_2017.shape[0]) * 100
porcentaje_duplicadas_2021 = (df_preguntas_interes_2021.duplicated(keep=False).sum() / df_preguntas_interes_2021.shape[0]) * 100

# Imprimir el resultado
print(f'Porcentaje de filas duplicadas 2017: {porcentaje_duplicadas_2017:.2f}%')
print(f'Porcentaje de filas duplicadas 2021: {porcentaje_duplicadas_2021:.2f}%')

Aunque existe un porcentaje relativamente importante de filas repetidas en los Data Frames, es importante notar que esto puede ser un indicio que la poblacion a la cual se le hizo la encuesta dio respuestas muy similares entre sí. Sin embargo, también podria indicar errores en la recopilación de datos, pues se podrían haber incluido datos repetidos accidentalmente o haber existido problemas en el proceso de carga de datos.
 
A pesar de esto, la encuesta no tiene un margen de error tan alto como para justificar que estas duplicaciones se deben al azar o al error estadístico o de contexto. De acuerdo a la docuentacion de la encuesta en la sección de **Procedimiento de muestreo**:

"*Para el cálculo de los tamaños de muestra se establecieron los siguientes parámetros para cada UPZ en Bogotá y para cada municipio de Cundinamarca: precisión esperada medida en términos del error estándar relativo igual a 7%, con un nivel de confiabilidad del 95%, para la prevalencia de alrededor del 10% y un efecto de diseño de 1,2.*" 

Asi pues, más bien esta situación se puede deber a que las respuestas dadas fueron identicas entre la poblacion para el subconjutno de preguntas elegidas y que esta poblacion puede compartir situaciones similares que condicionaron sus respuestas a al conjunto de preguntas de interes seleccionadas.  Por tanto, no es prudente, ni necesario, eliminar los registros duplicados, sino utilizarlos para hacer un analisis más exashustivo más adelante para definir las causas de esta situación.

### 1.3 Consistencia

La consistencia de los datos es definida en términos de la integridad de los datos entre diferentes filas o columnas de una fuente o varias fuentes.

En el caso de la encuesta multiproposito para el subconjunto de preguntas seleccionadas se espera que todas las respuestas sean de tipo numerico (ya sea int o float) pues las respuestas a estas preguntas son de selección multiple en la encuesta y las respuestas se representan con valores numericos.

In [ ]:
# Verificar si todas las filas contienen valores numéricos e ignorar columnas que si pueden tener valores no numericos:
todas_filas_numericas_2017 = df_preguntas_interes_2017.drop(columns=['LOCALIDAD_TEX']).applymap(lambda x: isinstance(x, (int, float))).all(axis=1)
todas_filas_numericas_2021 = df_preguntas_interes_2021.drop(columns=['NOMBRE_LOCALIDAD', 'NOMBRE_ESTRATO']).applymap(lambda x: isinstance(x, (int, float))).all(axis=1)

# Verificar si hay algún False en la Serie resultante
hay_falso_2017 = any(~todas_filas_numericas_2017)
hay_falso_2021 = any(~todas_filas_numericas_2021)

# Imprimir el resultado
print("¿Hay algún valor no numerico en las filas para el 2017?", hay_falso_2017)
print("¿Hay algún valor no numerico en las filas para el 2021?", hay_falso_2021)

Como se puede ver todas las filas que se suponen tener valores numericos, los tienen. Por tanto, son consistentes los datos con las repsuestas esperadas.

### 1.4 Validacion

### 1.4.1 Revisar que los valores asignados tengan sentido para la encuesta.

In [ ]:
import pandas as pd

# Supongamos que df_preguntas_interes_2017 es tu DataFrame

# Verificar si hay valores no positivos en cada columna (excluyendo 'LOCALIDAD_TEX')
valores_no_positivos = df_preguntas_interes_2017.drop(columns=['LOCALIDAD_TEX']).applymap(lambda x: x < 0)

# Imprimir los resultados por columna
for columna in valores_no_positivos.columns:
    valores_no_positivos_en_columna = valores_no_positivos[columna]
    if len(valores_no_positivos_en_columna[valores_no_positivos_en_columna].index.tolist())>0:
        print(f"Valores no positivos en la columna '{columna}':")


Como se puede ver todos los valores son positivos y, por tanto, son válidos y se ajustan a las respuestas esperadas de la encuesta.

In [ ]:
# Supongamos que df_preguntas_interes_2017 es tu DataFrame

# Verificar si hay valores no positivos en cada columna (excluyendo 'LOCALIDAD_TEX')
valores_no_positivos = df_preguntas_interes_2021.drop(columns=['NOMBRE_LOCALIDAD', 'NOMBRE_ESTRATO']).applymap(lambda x: x < 0)

# Imprimir los resultados por columna
for columna in valores_no_positivos.columns:
    valores_no_positivos_en_columna = valores_no_positivos[columna]
    if len(valores_no_positivos_en_columna[valores_no_positivos_en_columna].index.tolist())>0:
        print(f"Valores no positivos en la columna '{columna}':")

## 2. Entendimiento de los datos

### 2.1 Histogramas
En primer lugar, a continuacion se muestra un grafico de barras para mostrar cuales fueron el numero de personas con asma con respecto al numero total de los encuestados en 2017 y 2021, respectivamente.

In [6]:
# Obtener las variables únicas para el eje x y ordenarlas
Encuestados_2017 = len(df_preguntas_interes_2017)
Encuestados_2021 = len(df_preguntas_interes_2021)

Personas_asma_2017 = (df_preguntas_interes_2017['NPCFP14I'] == 1).sum()
Personas_asma_2021 = (df_preguntas_interes_2021['NPCFP14F'] == 1).sum()

print("Encuestados 2017: " + str(Encuestados_2017))
print("Personas con asma 2017: " + str(Personas_asma_2017))
print("Encuestados 2021: " + str(Encuestados_2021))
print("Personas con asma 2021: " + str(Personas_asma_2021))


NameError: name 'df_preguntas_interes_2017' is not defined

In [3]:
# Datos obtenidos
anos = ['2017', '2021']
encuestados = [encuestados_2017, encuestados_2021]
personas_asma = [personas_asma_2017, personas_asma_2021]

# Datos obtenidos
fechas = ['2017', '2021']
encuestados = [encuestados_2017, encuestados_2021]
personas_asma = [personas_asma_2017, personas_asma_2021]

# Crear la gráfica de barras apiladas
plt.figure(figsize=(10, 5))
plt.bar(fechas, encuestados, color='blue', label='Encuestados')
plt.bar(fechas, personas_asma, color='orange', label='Personas con Asma')
plt.xlabel('Año')
plt.ylabel('Número de Personas')
plt.title('Comparación de Personas con Asma respecto a Encuestados por Año')
plt.legend()
plt.show()

NameError: name 'encuestados_2017' is not defined

Ahora, se procederá a utilizar un gráfico de barras para mostrar las respuestas más comunes a la pregunta seleccionadas como relevante. Esto también sirve para verificar que ninguna de las respuetas a las preguntas, tengan un valor fuera del normal o esperado.

In [ ]:
# Supongamos que df_preguntas_interes_2017 y df_preguntas_interes_2021 son tus DataFrames

# Obtener las variables únicas para el eje x y ordenarlas
variables_2017 = df_preguntas_interes_2017['LOCALIDAD_TEX'].value_counts().index
variables_2021 = df_preguntas_interes_2021['NOMBRE_LOCALIDAD'].value_counts().index

# Crear una figura con dos subgráficos en una fila y dos columnas
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Primer subgráfico
axs[0].bar(variables_2017, df_preguntas_interes_2017['LOCALIDAD_TEX'].value_counts()[variables_2017], color='skyblue', edgecolor='black')
axs[0].set_title('Frecuencia de Localidades en 2017')
axs[0].set_xlabel('Localidades')
axs[0].set_ylabel('Frecuencia')
axs[0].tick_params(axis='x', rotation=90)  # Rotación y alineación horizontal a la derecha

# Segundo subgráfico
axs[1].bar(variables_2021, df_preguntas_interes_2021['NOMBRE_LOCALIDAD'].value_counts()[variables_2021], color='lightcoral', edgecolor='black')
axs[1].set_title('Frecuencia de Localidades en 2021')
axs[1].set_xlabel('Localidades')
axs[1].set_ylabel('Frecuencia')
axs[1].tick_params(axis='x', rotation=90)  # Rotación y alineación horizontal a la derecha

# Ajustar el diseño para evitar solapamiento de etiquetas
plt.tight_layout()

# Mostrar la figura con los dos subgráficos
plt.show()


In [ ]:
df_preguntas_interes_2017['LOCALIDAD_TEX'].value_counts()

In [ ]:
df_preguntas_interes_2021['NOMBRE_LOCALIDAD'].value_counts()

Como se puede ver en el gráfico de barras todos los valores para las localidades tienen valores que tienen sentido. En el 2017 y 2021 la mayoria de las personas encuestadas eran de la localidad de Suba.

In [ ]:
# Obtener las variables únicas para el eje x y ordenarlas
variables_2017 = df_preguntas_interes_2017['NVCBP11AA'].value_counts().index
variables_2021 = df_preguntas_interes_2021['NVCBP11AA'].value_counts().index

# Crear una figura con dos subgráficos en una fila y dos columnas
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Primer subgráfico
axs[0].bar(variables_2017, df_preguntas_interes_2017['NVCBP11AA'].value_counts()[variables_2017], color='skyblue', edgecolor='black')
axs[0].set_title('Frecuencia de Estratos en 2017')
axs[0].set_xlabel('Estrato')
axs[0].set_ylabel('Frecuencia')
axs[0].tick_params(axis='x', rotation=0)  

# Segundo subgráfico
axs[1].bar(variables_2021, df_preguntas_interes_2021['NVCBP11AA'].value_counts()[variables_2021], color='lightcoral', edgecolor='black')
axs[1].set_title('Frecuencia de Estratos en 2021')
axs[1].set_xlabel('Estrato')
axs[1].set_ylabel('Frecuencia')
axs[1].tick_params(axis='x', rotation=0)  

# Ajustar el diseño para evitar solapamiento de etiquetas
plt.tight_layout()

# Mostrar la figura con los dos subgráficos
plt.show()

Como se puede ver en el gráfico superior los estratos 2 y 3 son los más prevalentes en la encuesta (tanto del 2017 y 2021) mientras que los estratos 1 y 6 no son tan relevantes. Igualmente, los estratos 5 y 6 tampoco aportan mucho peso a la encuesta.

#### 2.2 Barras de Frecuencia
Con el objetivo de destacar de manera más efectiva las preguntas relevantes para el análisis, especialmente en casos donde una persona informa positivamente sobre una enfermedad respiratoria, se presenta un gráfico de barras. Este gráfico de frecuencia se construye asumiendo que la respuesta a alguna pregunta relacionada con enfermedades respiratorias fue afirmativa. La visualización busca resaltar de manera clara y concisa las tendencias y patrones específicos relacionados con este grupo de respuestas afirmativas, proporcionando así una perspectiva más enfocada en el contexto del análisis de enfermedades respiratorias.

Para la encuensta realizada en el 2017:

In [ ]:
# Filtrar el DataFrame para cuando la respuesta a NPCFP14I es 1, es decir la persona tiene asma
df_filtered = df_preguntas_interes_2017[df_preguntas_interes_2017['NPCFP14I'] == 1]

# Obtener las variables diferentes de 'NPCFP14I'
otras_variables = df_preguntas_interes_2017.columns[df_preguntas_interes_2017.columns != 'NPCFP14I']

# Calcular el número de filas y columnas necesario para mostrar las gráficas
num_filas = (len(otras_variables) + 2) // 3
num_columnas = 3

# Crear una figura con subgráficos
fig, axs = plt.subplots(num_filas, num_columnas, figsize=(15, num_filas * 5))

# Iterar sobre las otras variables y mostrar un gráfico de barras para cada una
for i, variable in enumerate(otras_variables):
    row = i // num_columnas
    col = i % num_columnas

    # Rotar las etiquetas del eje x solo en la primera fila
    rotation = 90 if row == 0 else 45

    sns.countplot(x=variable, data=df_filtered, ax=axs[row, col])
    axs[row, col].set_title(f'Relación con {variable} cuando NPCFP14I=1')
    axs[row, col].tick_params(axis='x', rotation=rotation)  # Ajustar la rotación

# Ajustar el diseño
plt.tight_layout()
plt.show()

In [ ]:
# Filtrar el DataFrame para cuando la respuesta a NPCFP14I es 1, es decir la persona tiene asma
df_filtered = df_preguntas_interes_2021[df_preguntas_interes_2021[COLUMNA_ASMA_2021] == 1]

# Obtener las variables diferentes de 'NPCFP14I'
otras_variables = df_preguntas_interes_2021.columns[df_preguntas_interes_2021.columns != COLUMNA_ASMA_2021]

# Calcular el número de filas y columnas necesario para mostrar las gráficas
num_filas = (len(otras_variables) + 2) // 3
num_columnas = 3

# Crear una figura con subgráficos
fig, axs = plt.subplots(num_filas, num_columnas, figsize=(15, num_filas * 5))

# Iterar sobre las otras variables y mostrar un gráfico de barras para cada una
for i, variable in enumerate(otras_variables):
    row = i // num_columnas
    col = i % num_columnas

    # Rotar las etiquetas del eje x solo en la primera fila
    rotation = 90 if row == 0 else 45

    sns.countplot(x=variable, data=df_filtered, ax=axs[row, col])
    axs[row, col].set_title(f'Relación con {variable} cuando NPCFP14F=1')
    axs[row, col].tick_params(axis='x', rotation=rotation)  # Ajustar la rotación

# Ajustar el diseño
plt.tight_layout()
plt.show()

### 2.3 Prueba de chi-cuadrado

Para examinar la asociación entre varias variables categóricas, se puede utilizar la prueba de independencia de chi-cuadrado de Pearson. Esta prueba evalúa si hay una relación significativa entre dos variables categóricas.

La prueba de chi-cuadrado (χ²) es una prueba estadística que se utiliza para determinar si hay una asociación significativa entre dos variables categóricas. La prueba evalúa si las observaciones empíricas, que están organizadas en una tabla de contingencia, difieren de las expectativas teóricas bajo la hipótesis nula de independencia entre las variables.


La prueba de chi-cuadrado (χ²) es una prueba estadística que se utiliza para determinar si hay una asociación significativa entre dos variables categóricas. La prueba evalúa si las observaciones empíricas, que están organizadas en una tabla de contingencia, difieren de las expectativas teóricas bajo la hipótesis nula de independencia entre las variables.

Aquí hay una descripción básica de cómo funciona la prueba de chi-cuadrado:

1. Hipótesis Nula (H₀): La hipótesis nula asume que no hay asociación entre las dos variables categóricas; son independientes.

2. Valor p: Se determina la probabilidad de obtener el estadístico de prueba observado (o uno más extremo) bajo la hipótesis nula.

3. Conclusión: Si el valor p es menor que el nivel de significancia elegido (de 0.05), se rechaza la hipótesis nula, lo que sugiere que hay una asociación significativa entre las variables.

En primer lugar, se hara este procedimiento para la encuesta realizada en el año 2017:

In [ ]:
nivel_de_significancia = 0.05

In [ ]:
from scipy.stats import chi2_contingency

# Seleccionar las columnas relevantes
columnas_seleccionadas = preguntas_interes_2017 + preguntas_comunes

# Crear una matriz para almacenar los p-values de chi-cuadrado
p_values_matrix = pd.DataFrame(index=['NPCFP14I'], columns=columnas_seleccionadas)

# Calcular las pruebas de chi-cuadrado y llenar la matriz
for col in columnas_seleccionadas:
    if col != 'NPCFP14I':
        tabla_contingencia = pd.crosstab(df_preguntas_interes_2017['NPCFP14I'], df_preguntas_interes_2017[col])
        chi2, p_value, _, _ = chi2_contingency(tabla_contingencia)
        p_values_matrix.at['NPCFP14I', col] = p_value
        print(" Valor p: ", p_value, " de NPCFP14I y ", col)

# Convertir p-values a números y establecer NaN en 1 (sin significancia)
p_values_matrix = p_values_matrix.apply(pd.to_numeric, errors='coerce').fillna(1)

# Crear un mapa de calor
plt.figure(figsize=(12, 4))
sns.heatmap(p_values_matrix, annot=True, cmap='coolwarm', fmt=".3f")
plt.title('Mapa de calor de correlación chi-cuadrado entre NPCFP14I y otras variables categóricas')
plt.show()

A partir del mapa de calor construido se analizará el valor p para cada una de las variables contruidas:


1. Valor p menor que el nivel de significancia (0.05):

    - Interpretación: Se rechaza la hipótesis nula.

    - Significado: Hay evidencia significativa para decir que existe una correlación significativa entre las variables categoricas en estudio.
    
2. Valor p mayor que el nivel de significancia (0.05):

    - Interpretación: No se rechaza la hipótesis nula.
    
    - Significado: No hay evidencia significativa para decir que existe una correlación significativa entre las variables categoricas en estudio.

In [ ]:
p_values_matrix


Construyendo la siguiente tabla se ilutra ilustra si el valor es menor al nivel de significancia o no:

In [ ]:
p_values_matrix = p_values_matrix.applymap(lambda x: x < nivel_de_significancia)
print(p_values_matrix.T)

Como se puede ver en la matriz superior NO existe una corelacion entre las siguientes variables categoricas:

1. NPCFP14I y DPTOMPIO: ¿Cuál es el material predominante de las paredes exteriores?

2. NPCFP14I y NVCBP12: Divipola departamento-municipio

3. NPCFP14I y NVCBP4: ¿La edificación está ubicada en un conjunto cerrado?


In [ ]:
# Seleccionar las columnas relevantes
columnas_seleccionadas = preguntas_interes_2021 + preguntas_comunes

# Crear una matriz para almacenar los p-values de chi-cuadrado
p_values_matrix = pd.DataFrame(index=['NPCFP14F'], columns=columnas_seleccionadas)

# Calcular las pruebas de chi-cuadrado y llenar la matriz
for col in columnas_seleccionadas:
    if col != 'NPCFP14F':
        tabla_contingencia = pd.crosstab(df_preguntas_interes_2021['NPCFP14F'], df_preguntas_interes_2021[col])
        _, p_value, _, _ = chi2_contingency(tabla_contingencia)
        p_values_matrix.at[COLUMNA_ASMA_2021, col] = p_value
        print("Valor p: ", p_value, " de NPCFP14F y ", col)

# Convertir p-values a números y establecer NaN en 1 (sin significancia)
p_values_matrix = p_values_matrix.apply(pd.to_numeric, errors='coerce').fillna(1)

# Crear un mapa de calor
plt.figure(figsize=(12, 4))
sns.heatmap(p_values_matrix, annot=True, cmap='coolwarm', fmt=".3f")
plt.title('Mapa de calor de correlación chi-cuadrado entre NPCFP14F y otras variables categóricas')
plt.show()

In [ ]:
p_values_matrix

In [ ]:
p_values_matrix = p_values_matrix.applymap(lambda x: x < nivel_de_significancia)
print(p_values_matrix.T)

Como se puede ver en la matriz superior NO existe una corelacion entre las siguientes variables categoricas:

1. NPCFP14F y DPTO: Departamento

2. NPCFP14F y MPIO: Municipio

3. NPCFP14F y NVCBP12: ¿Cuál es el material predominante de las paredes exteriores?

4. NPCFP14F y NVCBP4: ¿La edificación está ubicada en un conjunto residencial?

5. NPCFP14F y NVCBP10: Tipo de vivienda

Como se pudo ver en ambas encuestas de los años 2017 y 2021 coinciden en que las preguntas de: ¿Cuál es el material predominante de las paredes exteriores? (NVCBP12) y ¿La edificación está ubicada en un conjunto cerrado? (NVCBP4) no tienen corelacion alguna con las repsuestas dadas acerca de las enfermedades respiratorias en las respectivas encuentas. Del mismo modo, el municipio - departamento tampoco parce ser una variable relevante en la corelacion, esto puede deberse a falta de variabilidad en los datos.

Por otro lado, la gran mayoria de otras variables escogidas tienen una alta corelacion y deben de ser tomadas en cuenta para posteriores analisis y para el modelamiento del modelo multidimensional, creación del Data mart y proceso ETL. 